In [4]:
import numpy as np

def read_sparse_matrix_method2_crs(filename):
    with open(filename, "r") as f:
        n = int(f.readline().strip())  # Read dimension

        # Collect entries by row
        entries_by_row = [[] for _ in range(n)]

        for line in f:
            parts = line.strip().split(',')
            if len(parts) == 3:
                value, row, col = float(parts[0]), int(parts[1]), int(parts[2])
                entries_by_row[row].append((col, value))

        # Create CRS arrays
        values = []
        ind_col = []
        row_ptr = [0]

        # Track diagonal elements to verify none are zero
        diag_elements = [None] * n

        for i, row_entries in enumerate(entries_by_row):
            # Sort entries by column index
            row_entries.sort()

            # Combine entries with the same column index
            combined_entries = {}
            for col, val in row_entries:
                combined_entries[col] = combined_entries.get(col, 0) + val
                if col == i:  # Track diagonal element
                    diag_elements[i] = combined_entries[col]

            # Add entries to values and column indices
            for col, val in sorted(combined_entries.items()):
                values.append(val)
                ind_col.append(col)

            # Update row pointer
            row_ptr.append(len(values))

        # Check for missing or zero diagonal elements
        for i in range(n):
            if diag_elements[i] is None or abs(diag_elements[i]) < 1e-10:
                raise ValueError(f"Zero or missing diagonal element at row {i}. Matrix not suitable for Gauss-Seidel method.")

    return n, values, ind_col, row_ptr

def read_vector(filename):
    with open(filename, "r") as f:
        n = int(f.readline().strip())  # Read vector dimension
        b = np.array([float(f.readline().strip()) for _ in range(n)])  # Read values
    return b

def calculate_residual_crs(n, values, ind_col, row_ptr, x, b):
    """Calculate the residual ||Ax - b||∞ for a matrix in CRS format."""
    Ax = np.zeros(n)
    for i in range(n):
        row_start = row_ptr[i]
        row_end = row_ptr[i+1]
        for idx in range(row_start, row_end):
            col = ind_col[idx]
            Ax[i] += values[idx] * x[col]

    return np.max(np.abs(Ax - b))

def gauss_seidel_crs(n, values, ind_col, row_ptr, b, x, epsilon, max_iterations):
    """
    Gauss-Seidel method for solving Ax = b where A is in CRS format.
    Uses the one-vector approach and efficiently tracks convergence.
    """
    for iteration in range(max_iterations):
        max_diff = 0  # Track maximum change directly

        for i in range(n):
            x_old = x[i]  # Save only the current element

            row_start = row_ptr[i]
            row_end = row_ptr[i+1]

            sum_ax = 0.0
            diag_value = None

            for idx in range(row_start, row_end):
                col = ind_col[idx]
                value = values[idx]

                if col == i:
                    diag_value = value  # Diagonal element
                else:
                    sum_ax += value * x[col]  # Non-diagonal sum

            if diag_value is None or abs(diag_value) < 1e-10:
                raise ValueError(f"Zero diagonal element found at row {i}, cannot proceed with Gauss-Seidel.")

            x[i] = (b[i] - sum_ax) / diag_value  # Update x_i

            # Calculate change for this element only
            diff = abs(x[i] - x_old)
            max_diff = max(max_diff, diff)

        # Calculate current residual
        residual = calculate_residual_crs(n, values, ind_col, row_ptr, x, b)

        # Print errors for this iteration
        print(f"Iteration {iteration+1}: change in x = {max_diff:.2e}, residual = {residual:.2e}")

        # Check for convergence
        if residual < epsilon or max_diff < epsilon:
            print(f"Converged in {iteration+1} iterations.")
            return x

    print(f"Reached max iterations ({max_iterations}).")
    return x

def verify_solution_crs(n, values, ind_col, row_ptr, x_solution, b):
    """Verify the solution by computing ||Ax - b||∞."""
    error = calculate_residual_crs(n, values, ind_col, row_ptr, x_solution, b)
    print("Verification ||Ax - b||∞:", error)
    return error

# Process all matrix and vector files
matrix_files = ["a_1.txt", "a_2.txt", "a_3.txt", "a_4.txt"]
vector_files = ["b_1.txt", "b_2.txt", "b_3.txt", "b_4.txt"]

for a_file, b_file in zip(matrix_files, vector_files):
    print(f"Processing {a_file} and {b_file}...")
    n, values, ind_col, row_ptr = read_sparse_matrix_method2_crs(a_file)
    b = read_vector(b_file)
    x = np.zeros(n)  # Initial guess: all zeros
    epsilon = 1e-10  # Precision threshold
    max_iterations = 10000  # Safety limit

    # Solve using Gauss-Seidel
    x_solution = gauss_seidel_crs(n, values, ind_col, row_ptr, b, x, epsilon, max_iterations)

    # Verify solution
    error = verify_solution_crs(n, values, ind_col, row_ptr, x_solution, b)
    print("First 10 values of x:", x_solution[:10])
    print("-------------------------------")


Processing a_1.txt and b_1.txt...
Iteration 1: change in x = 1.10e+00, residual = 1.10e+02
Iteration 2: change in x = 9.91e-02, residual = 3.46e+00
Iteration 3: change in x = 3.12e-03, residual = 5.40e-02
Iteration 4: change in x = 5.01e-05, residual = 7.10e-04
Iteration 5: change in x = 6.58e-07, residual = 4.83e-06
Iteration 6: change in x = 4.48e-09, residual = 3.46e-08
Iteration 7: change in x = 3.26e-11, residual = 1.61e-10
Converged in 7 iterations.
Verification ||Ax - b||∞: 1.6075318853836507e-10
First 10 values of x: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
-------------------------------
Processing a_2.txt and b_2.txt...
Iteration 1: change in x = 6.36e-01, residual = 4.36e+01
Iteration 2: change in x = 4.54e-01, residual = 1.59e+01
Iteration 3: change in x = 2.32e-01, residual = 2.64e+00
Iteration 4: change in x = 6.21e-02, residual = 5.71e-01
Iteration 5: change in x = 1.29e-02, residual = 1.55e-01
Iteration 6: change in x = 4.02e-03, residual = 4.76e-02
Iteration 7: change in x = 1.